In [1]:
def dected_ROI(frame, width, height ,limit):   #保留frame [limit:height-limit, limit: width-limit] 範圍內的值  其餘弄成黑色
#     margin = limit//2
    mask=np.zeros([height,width],dtype=np.uint8) 
    mask[limit:height-limit, limit: width-limit] = 255
    image = cv2.bitwise_and(frame, frame, mask=mask) 
    return image

In [2]:
def detection_filter(detection_object):
    CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
    "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
    "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
    "sofa", "train", "tvmonitor"]
#當偵測到4個物體時，detection.shape[2] = 4，所以這裡是偵測到的個數
    #這是detections.shape = (1, 1, 4, 7)
    need_delete_index=[]

    for i in np.arange(0,detection_object.shape[0]):
        # extract the confidence (i.e., probability) associated
        # with the prediction
        confidence = detection_object[i, 2]
        # extract the index of the class label from the
        # detections list
        idx = int(detection_object[i, 1])

        #[0.         1.         0.30299687 0.07226107 0.03428271 0.6716727  0.9734354 ]
        #[0]=?  [1]=idx [2]=confidence [3:7] = corrdinate 
        # filter out weak detections by requiring a minimum
        # confidence
        
        if confidence < 0.3 or CLASSES[idx] != "person" :
            #print("delete:")
            #print(detection_object[i,:])
            need_delete_index.append(i)

    #print("need_delete_index")
    #print(need_delete_index)
    detection_object = np.delete(detection_object, need_delete_index, axis=0)

    return detection_object

In [3]:
def get_background(frame, vname):
    path = 'save/background_{}.jpg'.format(vname)
    cv2.imwrite(path, frame)

In [4]:

def main_func_predict(video_path, dected_per_frame = 30, method = 'dlib', 
                      seconds_proid = 10):
    getFrame = False


    
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    limit = int(max(width, height)/10)
#     width = 500
#     height = 1000
    dis_threshold = max(height, width)/10
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    net = cv2.dnn.readNetFromCaffe('mobilenet_ssd_caffe/MobileNetSSD_deploy.prototxt', 'mobilenet_ssd_caffe/MobileNetSSD_deploy.caffemodel')
    
#     videoWriter = cv2.VideoWriter(write_path, cv2.VideoWriter_fourcc(*'DIVX'), fps, (width, height))
    total_frame = 0
    sec = 0
    print("width:{}".format(width))
    print("height:{}".format(height))
    trks = mutiTracker.Trackers(dist_thresh = dis_threshold, max_frames_to_skip = 2, 
                                max_trace_length = 600, 
                                leave_limit = limit/2, 
                                frame_info = (width, height),
                               video_name = '0',
                               time_proid = seconds_proid)
    peroid_count = False
    while(cap.isOpened()):
        ret,frame = cap.read()
        if not ret:
            print("end of the video file...")
            break
#         frame = imutils.resize(frame,width=500)
        
        if method == 'dlib':
            dected_frame = frame
            dected_frame = cv2.cvtColor(dected_frame, cv2.COLOR_BGR2RGB)
        else:
            dected_frame = frame
            
        tensor_frame = dected_ROI(frame, width, height ,limit)
        
        
        if len(trks.get_trackers()) != 0:
            trks.predict_update(dected_frame)
            trackers = trks.get_trackers()
            for tracker in trackers:
                cen = tracker.centroid
                ID = tracker.track_id
                box = tracker.bbox
                cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]),(0, 255, 0), 1)
                num = cen[0]
                amend = num% 10
                cv2.putText(frame, str(ID), (cen[0]-(amend*1), cen[1]), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 255, 255), 1, cv2.LINE_AA)
            trks.checked_leaved()    
        
        if total_frame % dected_per_frame == 0:
            print("dected!!!!!!!!!")
            blob = cv2.dnn.blobFromImage(tensor_frame, 0.007843, (width, height), 127.5)
            net.setInput(blob)
            detections = net.forward()
            detections = np.squeeze(detections,axis=None)
            if(len(detections.shape)==1): #如果維度太低，改變為適當的維度
                detections = np.expand_dims(detections, axis=0)
            filtered_detections = detection_filter(detections)
            rects = list()
            for i in range (0,filtered_detections.shape[0]):
                bounding_box = filtered_detections[i, 3:7] * np.array([width, height, width, height])
                (startX, startY, endX, endY) = bounding_box.astype("int")
                rects.append((startX, startY, endX, endY))
#             rects, total_count, adds = ft.predict_Filter_people_box(np.squeeze(boxes), 
#                                                                     np.squeeze(classes).astype(np.int32),
#                                                                     np.squeeze(scores))
            for (startX, startY, endX, endY) in rects:
                cv2.rectangle(frame, (startX, startY), (endX, endY),(0, 0, 255), 2)
            trks.detect_Update(rects, dected_frame, method)

        W = width // 2
        cv2.line(frame, (W, 0), (W, height), (0, 0, 255), 5)
        trks.y_deraction()
        total_left  = trks.totalLeft
        total_right = trks.totalRight
        info = [
                ("Left", total_left),
                ("Right", total_right),
                ]
        for (i, (k, v)) in enumerate(info):
            text = "{}: {}".format(k, v)
            cv2.putText(frame, text, (10, height - ((i * 20) + 20)),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)            



        cv2.imshow('my_drawing', frame)
        total_frame+=1

        sec = int(round(total_frame/fps,3))
#                 print("sec{}".format(sec))

        if total_frame % 300 == 0:
            trks.save_left_right_count()
        print('-'*50)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    
    cap.release()
    cv2.destroyAllWindows()


In [5]:
import numpy as np
import time, re, imutils, glob, os, cv2
import tensorflow as tf
import numpy as np
from utils_v2 import filed_tool_v2, label_map_util, Filter, draw_tool_v2
from trackers import mutiTracker, correlation_tracker, kalman_filter, opencv_trackers

# detection_graph, category_index =set_model('rfcn_resnet101_coco_11_06_2017')
main_func_predict(1)

width:640
height:480
dected!!!!!!!!!
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
---------------------------------------------

 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
----------------

--------------------------------------------------
--------------------------------------------------
dected!!!!!!!!!
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
id:2 about to be delete !!
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
---------------

 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
dected!!!!!!!!!
track information  id:3 ,[371, 52, 494, 363] ,[432, 207], skip frame:2, confidencd:12.832275812547742
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
------------------------------------

--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
-------------------------------

...

 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
dected!!!!!!!!!
detections:[(447, 227, 498, 365)]
unusedCols:set()
unuseddefctions:[]
unusedCols after iou:set()
track information  id:5 ,(447, 227, 498, 

 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
----------------

 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
dected!!!!!!!!!
track information  id:5 ,[260, 205, 315, 391] ,[287, 298], skip frame:2, confidencd:17.114107121229836
track information  id:6 ,[129, 147, 243, 403] ,[186, 275], skip frame:1, confidencd:11.337201357130292
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
--------------------------------------------------
 in checked leave
-

--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
-------------------------------

--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
dected!!!!!!!!!
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
---------------